In [2]:
import utils.conf as conf
import preprocessing.data_cleaner as cleaner
import preprocessing.feature_extractor as extractor
%load_ext autoreload
%autoreload 2

In [3]:
train, test = cleaner.clean()

/home/wei/anaconda2/envs/pt27/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
/home/wei/anaconda2/envs/pt27/lib/python2.7/site-packages/numpy/lib/arraysetops.py:259: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]


In [4]:
train = extractor.aggreate_features(train)

In [5]:
test = extractor.aggreate_features(test)

In [17]:
import models.randomforest as randomforest

y_train = train['TripType']
X_train = train.drop(['TripType'],axis=1)
rf_clf = randomforest.RandomForest()
rf_clf.fit(X_train, y_train)


In [21]:
preds, classes = rf_clf.predict_proba(test)


In [34]:
column_names = ['VisitNumber']
for cls in classes:
    column_names.append('TripType_' + str(cls))


In [36]:
import pandas as pd
out = pd.DataFrame(columns=column_names)

In [38]:
out['VisitNumber'] = test['VisitNumber']


In [43]:
out.ix[:,'TripType_3':] = preds

In [47]:
out.to_csv('submission.csv', sep=',')

In [52]:
out2 = out.drop_duplicates()

In [55]:
test['VisitNumber'].drop_duplicates().size

95674

In [60]:
out3 = out.groupby('VisitNumber').mean()

In [64]:
out3.to_csv('submission.csv', sep=',')

In [66]:
out.head(5)

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0.12,0.26,0.38,0.10,0.08,0.00,0.02,0.00,0.0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.08,0.18,0.24,0.14,0.28,0.02,0.00,0.00,0.0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.08,0.16,0.20,0.16,0.28,0.04,0.00,0.00,0.0,...,0.00,0.02,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.14,0.20,0.28,0.08,0.22,0.00,0.00,0.00,0.0,...,0.00,0.00,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,0.04,0.16,0.26,0.18,0.08,0.00,0.00,0.06,0.0,...,0.02,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
out3.head(2)

,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,TripType_15,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0.105,0.200,0.275,0.120,0.215,0.015,0.005,0.00,0.0,0.015,...,0.00,0.005,0.005,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.030,0.145,0.285,0.145,0.095,0.000,0.010,0.04,0.0,0.140,...,0.02,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
